In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib.pyplot as plt
import matplotlib as mpl
font = {"family":"Ricty"}
mpl.rc('font', **font)

In [ ]:
# とりあえずデータ読み込め！
# 選挙結果のデータ
pres08 = pd.read_csv('officialSource/qss-inst-master/PREDICTION/betting-markets/data/pres08.csv')
# 世論調査のデータ
polls08 = pd.read_csv('officialSource/qss-inst-master/PREDICTION/betting-markets/data/polls08.csv')
# それぞれのデータ内に、オバマとマケインの得票率(もしくは支持率)の差を「margin」カラムとして保存
pres08['margin'] = pres08['Obama'] - pres08['McCain']
polls08['margin'] = polls08['Obama'] - polls08['McCain']

# 同じ州の同じ中間日に世論調査が行われているのか？を確認
polls08.groupby(['state','middate']).count()
# 行われてるので、平均をとる必要がある

# pythonでのDate型ってこうやるらしい
polls08['middate'] = pd.to_datetime(polls08['middate'])
# middate は最大でも20081101
polls08['middate'].max()

# 試しに色々計算させてみる
pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-11')
pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-16')
import math
abs(pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-16'))
type(pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-11'))

# 選挙日までの日数をカラムとして保存
polls08['DaysToElection'] = pd.to_datetime('2008-11-04') - polls08['middate']

# DaysToElectionを小さい方から並べたランクナンバーを振る
polls08 = polls08.assign(
    rankMin=polls08.groupby(by='state')['DaysToElection'].rank(method='min')
)
# 最も選挙日に近い世論調査に限定して、オバマとマケインの差の平均をとり、それを予測値カラム(カラム名：pred)として保存
poll = polls08[polls08['rankMin']==1].groupby(by='state', as_index=False).mean()[['state','margin']]
poll = poll.rename(columns={'margin':'pred'})
# 州の名前をつけ直す
poll = pd.merge(poll, pres08, how='left', on='state')
# 誤差カラムを追加
poll['error'] = poll['margin'] - poll['pred']
# 誤差の平均を算出(＝平均予測誤差)
poll['error'].mean()
# root mean squre(2乗平均平方根)で誤差を見てみる
math.sqrt((poll['error']**2).mean())